In [1]:
"""
Graph WaveNet / MTGNN-style model (adaptive adjacency + dilated gated TCN)
3-node graph trading model: ETH (target), BTC, ADA on 1-minute data.

This notebook implements:
- Multi-task training (Variant 3): CE (3-class) + exit_ret regression + soft utility loss
- Walk-forward CV with threshold selection on VAL and fixed evaluation on TEST
- Safer artifact saving:
    * .pt stores ONLY model weights (state_dict)
    * scalers stored as .npz (center/scale arrays, no pickle)
    * meta stored as .json (cfg, thresholds, fold indices, file names)
- Training logs include:
    * train/val total loss + components
    * val trade_auc and dir_auc each epoch (for selection/early stopping)
"""


'\nGraph WaveNet / MTGNN-style model (adaptive adjacency + dilated gated TCN)\n3-node graph trading model: ETH (target), BTC, ADA on 1-minute data.\n\nThis notebook implements:\n- Multi-task training (Variant 3): CE (3-class) + exit_ret regression + soft utility loss\n- Walk-forward CV with threshold selection on VAL and fixed evaluation on TEST\n- Safer artifact saving:\n    * .pt stores ONLY model weights (state_dict)\n    * scalers stored as .npz (center/scale arrays, no pickle)\n    * meta stored as .json (cfg, thresholds, fold indices, file names)\n- Training logs include:\n    * train/val total loss + components\n    * val trade_auc and dir_auc each epoch (for selection/early stopping)\n'

In [2]:
# ======================================================================
# Step 0: Imports, seed, config (single unified block)
# ======================================================================

import os
import json
import math
import random
import shutil
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler

from sklearn.preprocessing import RobustScaler
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, roc_auc_score


def seed_everything(seed: int = 1234) -> None:
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything(100)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("DEVICE:", DEVICE)

torch.set_num_threads(max(1, os.cpu_count() or 4))

# --- Global config (single unified block) ---
CFG: Dict[str, Any] = {
    # data
    "freq": "1min",
    "data_dir": "../dataset",
    "final_test_frac": 0.10,

    # order book
    "book_levels": 15,
    "top_levels": 5,
    "near_levels": 5,

    # walk-forward windows (in sample-space)
    "train_min_frac": 0.50,
    "val_window_frac": 0.10,
    "test_window_frac": 0.10,
    "step_window_frac": 0.10,

    # scaling
    "max_abs_feat": 10.0,
    "max_abs_edge": 6.0,

    # correlations / graph
    "corr_windows": [6 * 5, 12 * 5, 24 * 5, 48 * 5, 84 * 5],  # 30m,1h,2h,4h,7h
    "corr_lags": [0, 1, 2, 5],  # lead-lag (no leakage)
    "edges_mode": "all_pairs",  # "manual" | "all_pairs"
    "edges": [("ADA", "BTC"), ("ADA", "ETH"), ("ETH", "BTC")],  # used if edges_mode="manual"
    "add_self_loops": True,
    "edge_transform": "fisher",  # "none" | "fisher"
    "edge_scale": True,

    # triple-barrier
    "tb_horizon": 1 * 30,
    "lookback": 4 * 12 * 5,
    "tb_pt_mult": 1.2,
    "tb_sl_mult": 1.1,
    "tb_min_barrier": 0.001,
    "tb_max_barrier": 0.006,

    # training
    "batch_size": 128,
    "epochs": 20,
    "lr": 3e-4,
    "weight_decay": 5e-4,
    "grad_clip": 1.0,
    "dropout": 0.15,

    # stability tricks
    "label_smoothing": 0.02,
    "use_weighted_sampler": True,
    "use_onecycle": True,

    # --- Graph WaveNet core channels
    "gwn_residual_channels": 64,
    "gwn_dilation_channels": 64,
    "gwn_skip_channels": 128,
    "gwn_end_channels": 128,

    # blocks/layers: dilations reset each block
    "gwn_blocks": 3,
    "gwn_layers_per_block": 2,
    "gwn_kernel_size": 2,

    # adaptive adjacency
    "adj_emb_dim": 8,
    "adj_temperature": 1.0,
    "adaptive_topk": 3,  # for 3 nodes, 3 keeps all

    # adjacency regularization
    "adj_l1_lambda": 1e-3,
    "adj_prior_lambda": 1e-2,

    # build prior adjacency from edge_attr (last timestep)
    "prior_use_abs": False,
    "prior_diag_boost": 1.0,
    "prior_row_normalize": True,

    # trading eval
    "cost_bps": 1.0,

    # threshold sweep grids (val only)
    "thr_trade_grid": [0.50, 0.55, 0.60, 0.65, 0.70, 0.75],
    "thr_dir_grid":   [0.50, 0.55, 0.60, 0.65, 0.70],
    "eval_min_trades": 50,
    "max_trade_rate_val": 0.65,
    "trade_rate_penalty": 0.10,
    "thr_objective": "pnl_sum",
    "proxy_target_trades": [50, 100, 200],

    # selection metric
    "sel_metric_dir_weight": 0.50,  # selection: trade_auc + w*dir_auc

    # extra anti-overtrading penalty on probabilities (optional)
    "trade_prob_penalty": 0.01,  # penalize mean(p_short+p_long)

    # multi-task loss weights
    "loss_w_ce": 1.0,
    "loss_w_ret": 0.25,       # regression weight
    "loss_w_utility": 0.10,   # soft utility weight

    # regression / utility stability
    "exit_ret_clip": 0.03,    # clip exit_ret for ret/utility losses (log-return units)
    "ret_huber_delta": 0.01,  # Huber delta
    "utility_k": 2.0,         # tanh slope for soft position

    # artifact saving
    "artifact_dir": "./artifacts_gwnet_multitask",
}

ASSETS = ["ADA", "BTC", "ETH"]
ASSET2IDX = {a: i for i, a in enumerate(ASSETS)}
TARGET_ASSET = "ETH"
TARGET_NODE = ASSET2IDX[TARGET_ASSET]

ART_DIR = Path(CFG["artifact_dir"])
ART_DIR.mkdir(parents=True, exist_ok=True)

print("Assets:", ASSETS, "| Target:", TARGET_ASSET)
print("Artifacts dir:", str(ART_DIR.resolve()))


DEVICE: cpu
Assets: ['ADA', 'BTC', 'ETH'] | Target: ETH
Artifacts dir: /Users/vitalii/Desktop/Model_Market_Microstructure/Graph_Neural_Network_for_Market_Microstructure/TGNN2026/artifacts_gwnet_multitask


In [3]:
# ======================================================================
# Step 0.1: Graph edges
# ======================================================================

def build_edge_list(cfg: Dict[str, Any], assets: List[str]) -> List[Tuple[str, str]]:
    mode = str(cfg.get("edges_mode", "manual"))
    if mode == "manual":
        edges = list(cfg["edges"])
    elif mode == "all_pairs":
        edges = [(s, t) for s in assets for t in assets if s != t]
    else:
        raise ValueError(f"Unknown edges_mode={mode}")

    if bool(cfg.get("add_self_loops", True)):
        edges = edges + [(a, a) for a in assets]
    return edges


EDGE_LIST = build_edge_list(CFG, ASSETS)
EDGE_NAMES = [f"{s}->{t}" for (s, t) in EDGE_LIST]
EDGE_INDEX = torch.tensor([[ASSET2IDX[s], ASSET2IDX[t]] for (s, t) in EDGE_LIST], dtype=torch.long)

print("EDGE_LIST:", EDGE_NAMES)
print("EDGE_INDEX:", EDGE_INDEX.tolist())


EDGE_LIST: ['ADA->BTC', 'ADA->ETH', 'BTC->ADA', 'BTC->ETH', 'ETH->ADA', 'ETH->BTC', 'ADA->ADA', 'BTC->BTC', 'ETH->ETH']
EDGE_INDEX: [[0, 1], [0, 2], [1, 0], [1, 2], [2, 0], [2, 1], [0, 0], [1, 1], [2, 2]]


In [4]:
# ======================================================================
# Step 1: Data loading
# ======================================================================

def load_asset(asset: str, freq: str, data_dir: Path, book_levels: int, part: Tuple[int, int] = (0, 80)) -> pd.DataFrame:
    path = data_dir / f"{asset}_{freq}.csv"
    df = pd.read_csv(path)
    df = df.iloc[int(len(df) * part[0] / 100): int(len(df) * part[1] / 100)]

    df["timestamp"] = pd.to_datetime(df["system_time"]).dt.round("min")
    df = df.sort_values("timestamp").set_index("timestamp")

    bid_cols = [f"bids_notional_{i}" for i in range(book_levels)]
    ask_cols = [f"asks_notional_{i}" for i in range(book_levels)]

    needed = ["midpoint", "spread", "buys", "sells"] + bid_cols + ask_cols
    missing = [c for c in needed if c not in df.columns]
    if missing:
        raise ValueError(f"{asset}: missing columns in CSV: {missing[:10]}{'...' if len(missing) > 10 else ''}")

    return df[needed]


def load_all_assets() -> pd.DataFrame:
    freq = CFG["freq"]
    data_dir = Path(CFG["data_dir"])
    book_levels = CFG["book_levels"]

    def rename_cols(df_one: pd.DataFrame, asset: str) -> pd.DataFrame:
        rename_map = {
            "midpoint": asset,
            "buys": f"buys_{asset}",
            "sells": f"sells_{asset}",
            "spread": f"spread_{asset}",
        }
        for i in range(book_levels):
            rename_map[f"bids_notional_{i}"] = f"bids_vol_{asset}_{i}"
            rename_map[f"asks_notional_{i}"] = f"asks_vol_{asset}_{i}"
        return df_one.rename(columns=rename_map)

    df_ada = rename_cols(load_asset("ADA", freq, data_dir, book_levels, part=(0, 75)), "ADA")
    df_btc = rename_cols(load_asset("BTC", freq, data_dir, book_levels, part=(0, 75)), "BTC")
    df_eth = rename_cols(load_asset("ETH", freq, data_dir, book_levels, part=(0, 75)), "ETH")

    df = df_ada.join(df_btc).join(df_eth).reset_index()
    return df


df = load_all_assets()
for a in ASSETS:
    df[f"lr_{a}"] = np.log(df[a]).diff().fillna(0.0)

print("Loaded df:", df.shape)
print("Time range:", df["timestamp"].min(), "->", df["timestamp"].max())
print(df.head(2))


Loaded df: (12831, 106)
Time range: 2021-04-07 11:34:00+00:00 -> 2021-04-16 10:15:00+00:00
                  timestamp      ADA  spread_ADA      buys_ADA      sells_ADA  \
0 2021-04-07 11:34:00+00:00  1.16205      0.0001  56936.467913  258248.957367   
1 2021-04-07 11:35:00+00:00  1.16800      0.0022  56491.336799   78665.286640   

   bids_vol_ADA_0  bids_vol_ADA_1  bids_vol_ADA_2  bids_vol_ADA_3  \
0      876.869995     5984.169922        5.810000       18.240000   
1    33769.671875    23137.169922      550.299988      550.299988   

   bids_vol_ADA_4  ...  asks_vol_ETH_8  asks_vol_ETH_9  asks_vol_ETH_10  \
0    19844.640625  ...      373.700012      196.699997      2059.709961   
1    19012.320312  ...     3873.709961     1954.630005       197.039993   

   asks_vol_ETH_11  asks_vol_ETH_12  asks_vol_ETH_13  asks_vol_ETH_14  \
0      3874.989990      5901.209961       178.289993     28512.160156   
1     12661.990234     20006.970703     28562.310547      3874.379883   

     lr_ADA

In [5]:
# ======================================================================
# Step 2: Edge features (rolling corr with lead-lag)
# ======================================================================

def _fisher_z(x: np.ndarray, eps: float = 1e-6) -> np.ndarray:
    x = np.clip(x, -0.999, 0.999)
    return 0.5 * np.log((1.0 + x + eps) / (1.0 - x + eps))


def build_corr_array(
    df_: pd.DataFrame,
    corr_windows: List[int],
    edges: List[Tuple[str, str]],
    lags: List[int],
    transform: str = "fisher",
) -> np.ndarray:
    """
    Edge features per time:
      for edge s->t:
        for lag in lags:
          corr(lr_s.shift(lag), lr_t) over rolling window
    No leakage: shift(lag>0) uses past of source.
    Self-loop edges a->a: constant 1.0.
    """
    T_ = len(df_)
    E_ = len(edges)
    W_ = len(corr_windows)
    Lg = len(lags)
    out = np.zeros((T_, E_, W_ * Lg), dtype=np.float32)

    lr_map = {a: df_[f"lr_{a}"].astype(float) for a in ASSETS}

    for ei, (s, t) in enumerate(edges):
        if s == t:
            out[:, ei, :] = 1.0
            continue

        src0 = lr_map[s]
        dst0 = lr_map[t]

        feat_idx = 0
        for lag in lags:
            src = src0.shift(int(lag)) if int(lag) > 0 else src0

            for w in corr_windows:
                r = src.rolling(int(w), min_periods=1).corr(dst0)
                r = np.nan_to_num(r.to_numpy(dtype=np.float32), nan=0.0, posinf=0.0, neginf=0.0)
                if transform == "fisher":
                    r = _fisher_z(r).astype(np.float32)
                out[:, ei, feat_idx] = r
                feat_idx += 1

    return out.astype(np.float32)


edge_feat = build_corr_array(
    df,
    CFG["corr_windows"],
    EDGE_LIST,
    CFG["corr_lags"],
    transform=str(CFG.get("edge_transform", "fisher")),
)

print("edge_feat shape:", edge_feat.shape, "(T,E,edge_dim)")
print("edge_dim =", edge_feat.shape[-1])
print("Edge names:", EDGE_NAMES[: min(10, len(EDGE_NAMES))], "...")
print("edge_feat stats: mean=", float(edge_feat.mean()), "std=", float(edge_feat.std()))


edge_feat shape: (12831, 9, 20) (T,E,edge_dim)
edge_dim = 20
Edge names: ['ADA->BTC', 'ADA->ETH', 'BTC->ADA', 'BTC->ETH', 'ETH->ADA', 'ETH->BTC', 'ADA->ADA', 'BTC->BTC', 'ETH->ETH'] ...
edge_feat stats: mean= 0.4511896073818207 std= 0.500860333442688


In [6]:
# ======================================================================
# Step 3: Triple-barrier labels
# ======================================================================

def triple_barrier_labels_from_lr(
    lr: pd.Series,
    horizon: int,
    vol_window: int,
    pt_mult: float,
    sl_mult: float,
    min_barrier: float,
    max_barrier: float,
) -> Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    """
    Returns:
      y_tb: {0=down, 1=flat/no-trade, 2=up}
      exit_ret: realized log-return to exit (tp/sl/timeout)
      exit_t: exit index
      thr: barrier per t (float array, len T)
    No leakage: vol is shift(1).
    """
    lr = lr.astype(float).copy()
    T = len(lr)

    vol = lr.rolling(vol_window, min_periods=max(10, vol_window // 10)).std().shift(1)
    thr = (vol * np.sqrt(horizon)).clip(lower=min_barrier, upper=max_barrier)

    y = np.ones(T, dtype=np.int64)
    exit_ret = np.zeros(T, dtype=np.float32)
    exit_t = np.arange(T, dtype=np.int64)

    lr_np = lr.fillna(0.0).to_numpy(dtype=np.float64)
    thr_np = thr.fillna(min_barrier).to_numpy(dtype=np.float64)

    for t in range(T - horizon - 1):
        up = pt_mult * thr_np[t]
        dn = -sl_mult * thr_np[t]

        cum = 0.0
        hit = 1
        et = t + horizon
        er = 0.0

        for dt in range(1, horizon + 1):
            cum += lr_np[t + dt]
            if cum >= up:
                hit, et, er = 2, t + dt, cum
                break
            if cum <= dn:
                hit, et, er = 0, t + dt, cum
                break

        if hit == 1:
            er = float(np.sum(lr_np[t + 1: t + horizon + 1]))
            et = t + horizon

        y[t] = hit
        exit_ret[t] = er
        exit_t[t] = et

    return y, exit_ret, exit_t, thr_np


y_tb, exit_ret, exit_t, tb_thr = triple_barrier_labels_from_lr(
    df["lr_ETH"],
    horizon=CFG["tb_horizon"],
    vol_window=CFG["lookback"],
    pt_mult=CFG["tb_pt_mult"],
    sl_mult=CFG["tb_sl_mult"],
    min_barrier=CFG["tb_min_barrier"],
    max_barrier=CFG["tb_max_barrier"],
)

y_trade = (y_tb != 1).astype(np.int64)  # 1=trade, 0=no-trade

dist = np.bincount(y_tb, minlength=3)
print("TB dist [down,flat,up]:", dist)
print("Trade ratio (true):", float(y_trade.mean()))


TB dist [down,flat,up]: [2875 7413 2543]
Trade ratio (true): 0.42225859247135844


In [7]:
# ======================================================================
# Step 4: Node features
# ======================================================================

EPS = 1e-6


def safe_log1p(x: np.ndarray) -> np.ndarray:
    return np.log1p(np.maximum(x, 0.0))


def build_node_tensor(df_: pd.DataFrame) -> Tuple[np.ndarray, List[str]]:
    """
    Features per asset:
      lr, spread,
      log_buys, log_sells, ofi,
      DI_15,
      DI_L0..DI_L4,
      near_ratio_bid, near_ratio_ask,
      di_near, di_far
    """
    book_levels = CFG["book_levels"]
    top_k = CFG["top_levels"]
    near_k = CFG["near_levels"]

    if near_k >= book_levels:
        raise ValueError("CFG['near_levels'] must be < CFG['book_levels']")

    feat_names = [
        "lr", "spread",
        "log_buys", "log_sells", "ofi",
        "DI_15",
        "DI_L0", "DI_L1", "DI_L2", "DI_L3", "DI_L4",
        "near_ratio_bid", "near_ratio_ask",
        "di_near", "di_far",
    ]

    feats_all = []
    for a in ASSETS:
        lr = df_[f"lr_{a}"].values.astype(np.float32)
        spread = df_[f"spread_{a}"].values.astype(np.float32)

        buys = df_[f"buys_{a}"].values.astype(np.float32)
        sells = df_[f"sells_{a}"].values.astype(np.float32)

        log_buys = safe_log1p(buys).astype(np.float32)
        log_sells = safe_log1p(sells).astype(np.float32)

        ofi = ((buys - sells) / (buys + sells + EPS)).astype(np.float32)

        bids_lvls = np.stack([df_[f"bids_vol_{a}_{i}"].values.astype(np.float32) for i in range(book_levels)], axis=1)
        asks_lvls = np.stack([df_[f"asks_vol_{a}_{i}"].values.astype(np.float32) for i in range(book_levels)], axis=1)

        bid_sum = bids_lvls.sum(axis=1)
        ask_sum = asks_lvls.sum(axis=1)
        di_15 = ((bid_sum - ask_sum) / (bid_sum + ask_sum + EPS)).astype(np.float32)

        di_levels = []
        for i in range(top_k):
            b = bids_lvls[:, i]
            s = asks_lvls[:, i]
            di_levels.append(((b - s) / (b + s + EPS)).astype(np.float32))
        di_l0_4 = np.stack(di_levels, axis=1)  # (T,5)

        bid_near = bids_lvls[:, :near_k].sum(axis=1)
        ask_near = asks_lvls[:, :near_k].sum(axis=1)
        bid_far = bids_lvls[:, near_k:].sum(axis=1)
        ask_far = asks_lvls[:, near_k:].sum(axis=1)

        near_ratio_bid = (bid_near / (bid_far + EPS)).astype(np.float32)
        near_ratio_ask = (ask_near / (ask_far + EPS)).astype(np.float32)

        di_near = ((bid_near - ask_near) / (bid_near + ask_near + EPS)).astype(np.float32)
        di_far = ((bid_far - ask_far) / (bid_far + ask_far + EPS)).astype(np.float32)

        Xa = np.column_stack([
            lr, spread,
            log_buys, log_sells, ofi,
            di_15,
            di_l0_4[:, 0], di_l0_4[:, 1], di_l0_4[:, 2], di_l0_4[:, 3], di_l0_4[:, 4],
            near_ratio_bid, near_ratio_ask,
            di_near, di_far,
        ]).astype(np.float32)

        feats_all.append(Xa)

    X = np.stack(feats_all, axis=1).astype(np.float32)  # (T,N,F)
    return X, feat_names


X_node_raw, node_feat_names = build_node_tensor(df)

T = len(df)
L = CFG["lookback"]
H = CFG["tb_horizon"]

t_min = L - 1
t_max = T - H - 2
sample_t = np.arange(t_min, t_max + 1)
n_samples = len(sample_t)

print("X_node_raw:", X_node_raw.shape, "edge_feat:", edge_feat.shape)
print("n_samples:", n_samples, "| t range:", int(sample_t[0]), "->", int(sample_t[-1]))


X_node_raw: (12831, 3, 15) edge_feat: (12831, 9, 20)
n_samples: 12561 | t range: 239 -> 12799


In [8]:
# ======================================================================
# Step 5: Splits
# ======================================================================

def make_final_holdout_split(n_samples_: int, final_test_frac: float) -> Tuple[np.ndarray, np.ndarray]:
    if not (0.0 < final_test_frac < 0.5):
        raise ValueError("final_test_frac should be in (0, 0.5)")
    n_final = max(1, int(round(final_test_frac * n_samples_)))
    n_cv = n_samples_ - n_final
    if n_cv <= 50:
        raise ValueError("Too few samples left for CV after holdout split.")
    idx_cv = np.arange(0, n_cv, dtype=np.int64)
    idx_final = np.arange(n_cv, n_samples_, dtype=np.int64)
    return idx_cv, idx_final


def make_walk_forward_splits(
    n_samples_: int,
    train_min_frac: float,
    val_window_frac: float,
    test_window_frac: float,
    step_window_frac: float,
) -> List[Tuple[np.ndarray, np.ndarray, np.ndarray]]:
    train_min = int(train_min_frac * n_samples_)
    val_w = max(1, int(val_window_frac * n_samples_))
    test_w = max(1, int(test_window_frac * n_samples_))
    step_w = max(1, int(step_window_frac * n_samples_))

    splits = []
    start = train_min
    while True:
        tr_end = start
        va_end = tr_end + val_w
        te_end = va_end + test_w
        if te_end > n_samples_:
            break

        idx_train = np.arange(0, tr_end, dtype=np.int64)
        idx_val = np.arange(tr_end, va_end, dtype=np.int64)
        idx_test = np.arange(va_end, te_end, dtype=np.int64)
        splits.append((idx_train, idx_val, idx_test))

        start += step_w

    return splits


idx_cv_all, idx_final_test = make_final_holdout_split(n_samples, CFG["final_test_frac"])
n_samples_cv = len(idx_cv_all)

walk_splits = make_walk_forward_splits(
    n_samples_=n_samples_cv,
    train_min_frac=CFG["train_min_frac"],
    val_window_frac=CFG["val_window_frac"],
    test_window_frac=CFG["test_window_frac"],
    step_window_frac=CFG["step_window_frac"],
)

print("Holdout split:")
print(f"  n_samples total: {n_samples}")
print(f"  n_samples CV   : {len(idx_cv_all)}")
print(f"  n_samples FINAL: {len(idx_final_test)}")
print("\nWalk-forward folds:", len(walk_splits))
for i, (a, b, c) in enumerate(walk_splits, 1):
    print(f"  fold {i}: train={len(a)} | val={len(b)} | test={len(c)}")


Holdout split:
  n_samples total: 12561
  n_samples CV   : 11305
  n_samples FINAL: 1256

Walk-forward folds: 4
  fold 1: train=5652 | val=1130 | test=1130
  fold 2: train=6782 | val=1130 | test=1130
  fold 3: train=7912 | val=1130 | test=1130
  fold 4: train=9042 | val=1130 | test=1130


In [9]:
# ======================================================================
# Step 6: Dataset, scaling helpers, sampler
# ======================================================================

class LobGraphSequenceDataset3Class(Dataset):
    """
    Returns:
      x_seq:    (L,N,F)
      e_seq:    (L,E,D)
      y_tb:     scalar in {0,1,2} (SHORT, FLAT, LONG)
      exit_ret: scalar (log-return to exit)
      sidx:     scalar sample index (for time-order reconstruction)
    """
    def __init__(
        self,
        X_node: np.ndarray,
        E_feat: np.ndarray,
        y_tb_arr: np.ndarray,
        exit_ret_arr: np.ndarray,
        sample_t_: np.ndarray,
        indices: np.ndarray,
        lookback: int,
    ):
        self.X_node = X_node
        self.E_feat = E_feat
        self.y_tb = y_tb_arr
        self.exit_ret = exit_ret_arr
        self.sample_t = sample_t_
        self.indices = indices.astype(np.int64)
        self.L = int(lookback)

    def __len__(self) -> int:
        return int(len(self.indices))

    def __getitem__(self, i: int):
        sidx = int(self.indices[i])
        t = int(self.sample_t[sidx])
        t0 = t - self.L + 1

        x_seq = self.X_node[t0:t + 1]  # (L,N,F)
        e_seq = self.E_feat[t0:t + 1]  # (L,E,D)
        y = int(self.y_tb[t])
        er = float(self.exit_ret[t])

        return (
            torch.from_numpy(x_seq),
            torch.from_numpy(e_seq),
            torch.tensor(y, dtype=torch.long),
            torch.tensor(er, dtype=torch.float32),
            torch.tensor(sidx, dtype=torch.long),
        )


def collate_fn_3class(batch):
    xs, es, ys, ers, sidxs = zip(*batch)
    return (
        torch.stack(xs, 0),    # (B,L,N,F)
        torch.stack(es, 0),    # (B,L,E,D)
        torch.stack(ys, 0),    # (B,)
        torch.stack(ers, 0),   # (B,)
        torch.stack(sidxs, 0), # (B,)
    )


def make_weighted_sampler_3class(y_np: np.ndarray) -> WeightedRandomSampler:
    y_np = np.asarray(y_np, dtype=np.int64)
    counts = np.bincount(y_np, minlength=3).astype(np.float64)
    counts = np.maximum(counts, 1.0)
    class_w = counts.sum() / (3.0 * counts)
    sample_w = class_w[y_np].astype(np.float64)
    sample_w = torch.tensor(sample_w, dtype=torch.double)
    return WeightedRandomSampler(weights=sample_w, num_samples=len(sample_w), replacement=True)


def make_ce_weights_3class(y_np: np.ndarray) -> torch.Tensor:
    y_np = np.asarray(y_np, dtype=np.int64)
    counts = np.bincount(y_np, minlength=3).astype(np.float64)
    counts = np.maximum(counts, 1.0)
    w = counts.sum() / (3.0 * counts)
    return torch.tensor(w, dtype=torch.float32, device=DEVICE)


def fit_scale_nodes_train_only(
    X_node_raw_: np.ndarray,
    sample_t_: np.ndarray,
    idx_train: np.ndarray,
    max_abs: float = 10.0
) -> Tuple[np.ndarray, Dict[str, Any]]:
    """
    Fit RobustScaler on train timeline only. Returns scaled X and scaler params dict.
    """
    last_train_t = int(sample_t_[int(idx_train[-1])])
    train_time_mask = np.arange(0, last_train_t + 1)

    X_train_time = X_node_raw_[train_time_mask]  # (Ttr,N,F)
    _, _, Fdim = X_train_time.shape

    scaler = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(5.0, 95.0))
    scaler.fit(X_train_time.reshape(-1, Fdim))

    X_scaled = scaler.transform(X_node_raw_.reshape(-1, Fdim)).reshape(X_node_raw_.shape).astype(np.float32)
    X_scaled = np.clip(X_scaled, -max_abs, max_abs).astype(np.float32)
    X_scaled = np.nan_to_num(X_scaled, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)

    params = {
        "center_": scaler.center_.astype(np.float32),
        "scale_": scaler.scale_.astype(np.float32),
        "max_abs": float(max_abs),
    }
    return X_scaled, params


def fit_scale_edges_train_only(
    E_raw_: np.ndarray,
    sample_t_: np.ndarray,
    idx_train: np.ndarray,
    max_abs: float = 6.0
) -> Tuple[np.ndarray, Dict[str, Any]]:
    """
    Fit RobustScaler on train timeline only for edge features. Returns scaled E and scaler params dict.
    """
    last_train_t = int(sample_t_[int(idx_train[-1])])
    train_time_mask = np.arange(0, last_train_t + 1)

    E_train_time = E_raw_[train_time_mask]  # (Ttr,E,D)
    _, _, D = E_train_time.shape

    scaler = RobustScaler(with_centering=True, with_scaling=True, quantile_range=(5.0, 95.0))
    scaler.fit(E_train_time.reshape(-1, D))

    E_scaled = scaler.transform(E_raw_.reshape(-1, D)).reshape(E_raw_.shape).astype(np.float32)
    E_scaled = np.clip(E_scaled, -max_abs, max_abs).astype(np.float32)
    E_scaled = np.nan_to_num(E_scaled, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)

    params = {
        "center_": scaler.center_.astype(np.float32),
        "scale_": scaler.scale_.astype(np.float32),
        "max_abs": float(max_abs),
    }
    return E_scaled, params


def apply_scaler_params(X: np.ndarray, params: Dict[str, Any]) -> np.ndarray:
    """
    Apply stored RobustScaler params (center_, scale_) + clip.
    """
    center = np.asarray(params["center_"], dtype=np.float32)
    scale = np.asarray(params["scale_"], dtype=np.float32)
    max_abs = float(params["max_abs"])

    # Broadcast over last dim
    X2 = (X - center) / (scale + 1e-12)
    X2 = np.clip(X2, -max_abs, max_abs)
    X2 = np.nan_to_num(X2, nan=0.0, posinf=0.0, neginf=0.0).astype(np.float32)
    return X2


def split_trade_ratio(indices: np.ndarray, sample_t_: np.ndarray, y_trade_arr: np.ndarray) -> float:
    tt = sample_t_[indices]
    return float(y_trade_arr[tt].mean()) if len(tt) else float("nan")


In [10]:
# ======================================================================
# Step 7: Graph WaveNet model (multi-task: logits + ret_hat)
# ======================================================================

def build_static_adjacency_from_edges(edge_index: torch.Tensor, n_nodes: int, eps: float = 1e-8) -> torch.Tensor:
    """
    Build A_static (N,N), row-normalized, using the presence of edges in EDGE_LIST.
    """
    A = torch.zeros((n_nodes, n_nodes), dtype=torch.float32)
    src = edge_index[:, 0].long()
    dst = edge_index[:, 1].long()
    A[src, dst] = 1.0
    A = A / (A.sum(dim=-1, keepdim=True) + eps)
    return A


def build_adj_prior_from_edge_attr(
    edge_attr_last: torch.Tensor,    # (B,E,D)
    edge_index: torch.Tensor,        # (E,2) [src,dst]
    n_nodes: int,
    use_abs: bool = False,
    diag_boost: float = 1.0,
    row_normalize: bool = True,
    eps: float = 1e-8
) -> torch.Tensor:
    """
    Build A_prior (B,N,N) from edge_attr at the last timestep.
      w = sigmoid(mean(edge_attr)) in [0,1]
    """
    edge_attr_last = torch.nan_to_num(edge_attr_last, nan=0.0, posinf=0.0, neginf=0.0)
    B, E, D = edge_attr_last.shape
    r = edge_attr_last.mean(dim=-1)  # (B,E)
    if use_abs:
        r = r.abs()
    w = torch.sigmoid(r)  # (B,E)

    A = torch.zeros((B, n_nodes, n_nodes), device=edge_attr_last.device, dtype=edge_attr_last.dtype)
    src = edge_index[:, 0].to(edge_attr_last.device)
    dst = edge_index[:, 1].to(edge_attr_last.device)
    A[:, src, dst] = w

    diag = torch.arange(n_nodes, device=edge_attr_last.device)
    A[:, diag, diag] = torch.maximum(A[:, diag, diag], torch.full_like(A[:, diag, diag], float(diag_boost)))

    if row_normalize:
        A = A / (A.sum(dim=-1, keepdim=True) + eps)

    return torch.nan_to_num(A, nan=0.0, posinf=0.0, neginf=0.0)


class AdaptiveAdjacency(nn.Module):
    """
    Graph WaveNet-style adaptive adjacency from node embeddings:
      logits = relu(E1 @ E2^T) / temp
      (optional) top-k per row
      A_adapt = softmax(logits_row)
    """
    def __init__(self, n_nodes: int, cfg: Dict[str, Any]):
        super().__init__()
        self.n = int(n_nodes)
        k = int(cfg.get("adj_emb_dim", 8))
        self.E1 = nn.Parameter(0.01 * torch.randn(self.n, k))
        self.E2 = nn.Parameter(0.01 * torch.randn(self.n, k))
        temp = float(cfg.get("adj_temperature", 1.0))
        self.temp = max(temp, 1e-3)
        self.topk = int(cfg.get("adaptive_topk", self.n))

    def forward(self) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        logits = (self.E1 @ self.E2.t())
        logits = F.relu(logits) / self.temp  # (N,N)

        if self.topk is not None and 0 < self.topk < self.n:
            vals, idx = torch.topk(logits, k=self.topk, dim=-1)
            mask = torch.full_like(logits, fill_value=float("-inf"))
            mask.scatter_(-1, idx, vals)
            logits = mask

        A = torch.softmax(logits, dim=-1)  # row-stochastic
        sparsity_proxy = torch.sigmoid(logits)
        return A, sparsity_proxy, logits


class LearnableSupportMix(nn.Module):
    """
    Blend supports (static, prior, adapt) using softmax weights.
    """
    def __init__(self, n_supports: int = 3):
        super().__init__()
        self.w_logits = nn.Parameter(torch.zeros(n_supports, dtype=torch.float32))

    def forward(self) -> torch.Tensor:
        return torch.softmax(self.w_logits, dim=0)


class CausalConv2dTime(nn.Module):
    """
    2D convolution causal along time dimension only.
    Input:  (B,C,N,T)
    Conv kernel: (1,k), dilation: (1,d)
    """
    def __init__(self, in_ch: int, out_ch: int, kernel_size: int, dilation: int):
        super().__init__()
        self.k = int(kernel_size)
        self.d = int(dilation)
        self.conv = nn.Conv2d(in_ch, out_ch, kernel_size=(1, self.k), dilation=(1, self.d))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        pad_left = (self.k - 1) * self.d
        x = F.pad(x, (pad_left, 0, 0, 0))
        return self.conv(x)


def graph_message_passing(x: torch.Tensor, A: torch.Tensor) -> torch.Tensor:
    """
    x: (B,C,N,T)
    A: (B,N,N) with A[src,dst]
    returns: (B,C,N,T) where dst aggregates from src
    """
    return torch.einsum("bcnt,bnm->bcmt", x, A)


class GraphWaveNetBlock(nn.Module):
    def __init__(self, residual_ch: int, dilation_ch: int, skip_ch: int, kernel_size: int, dilation: int, dropout: float):
        super().__init__()
        self.filter_conv = CausalConv2dTime(residual_ch, dilation_ch, kernel_size=kernel_size, dilation=dilation)
        self.gate_conv = CausalConv2dTime(residual_ch, dilation_ch, kernel_size=kernel_size, dilation=dilation)

        self.residual_conv = nn.Conv2d(dilation_ch, residual_ch, kernel_size=(1, 1))
        self.skip_conv = nn.Conv2d(dilation_ch, skip_ch, kernel_size=(1, 1))

        self.dropout = nn.Dropout(float(dropout))
        self.bn = nn.BatchNorm2d(residual_ch)

    def forward(self, x: torch.Tensor, A: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        x = torch.nan_to_num(x, nan=0.0, posinf=0.0, neginf=0.0)
        A = torch.nan_to_num(A, nan=0.0, posinf=0.0, neginf=0.0)

        residual = x

        f = torch.tanh(self.filter_conv(x))
        g = torch.sigmoid(self.gate_conv(x))
        z = f * g  # (B,dilation_ch,N,T)

        z = self.dropout(z)

        skip = self.skip_conv(z)  # (B,skip_ch,N,T)

        out = self.residual_conv(z)  # (B,residual_ch,N,T)
        out = graph_message_passing(out, A)
        out = out + residual
        out = self.bn(out)

        out = torch.nan_to_num(out, nan=0.0, posinf=0.0, neginf=0.0)
        skip = torch.nan_to_num(skip, nan=0.0, posinf=0.0, neginf=0.0)
        return out, skip


class GraphWaveNetMultiTask(nn.Module):
    """
    Input:
      x_seq: (B,L,N,F)
      e_seq: (B,L,E,D)   (only used to build A_prior from last step)

    Outputs for target node only:
      logits:  (B,3)  classes [SHORT, FLAT, LONG]
      ret_hat: (B,)   predicted exit_ret (regression head)
    """
    def __init__(self, node_in: int, edge_dim: int, cfg: Dict[str, Any], n_nodes: int, target_node: int):
        super().__init__()
        self.cfg = cfg
        self.n_nodes = int(n_nodes)
        self.target_node = int(target_node)

        residual_ch = int(cfg["gwn_residual_channels"])
        dilation_ch = int(cfg["gwn_dilation_channels"])
        skip_ch = int(cfg["gwn_skip_channels"])
        end_ch = int(cfg["gwn_end_channels"])
        k = int(cfg["gwn_kernel_size"])
        blocks = int(cfg["gwn_blocks"])
        layers_per_block = int(cfg["gwn_layers_per_block"])
        drop = float(cfg.get("dropout", 0.0))

        self.in_proj = nn.Linear(int(node_in), residual_ch)

        A_static = build_static_adjacency_from_edges(EDGE_INDEX, n_nodes=self.n_nodes)
        self.register_buffer("A_static", A_static)

        self.adapt = AdaptiveAdjacency(n_nodes=self.n_nodes, cfg=cfg)
        self.support_mix = LearnableSupportMix(n_supports=3)

        self.blocks = nn.ModuleList()
        for _b in range(blocks):
            for l in range(layers_per_block):
                dilation = 2 ** l
                self.blocks.append(GraphWaveNetBlock(
                    residual_ch=residual_ch,
                    dilation_ch=dilation_ch,
                    skip_ch=skip_ch,
                    kernel_size=k,
                    dilation=dilation,
                    dropout=drop,
                ))

        self.end1 = nn.Conv2d(skip_ch, end_ch, kernel_size=(1, 1))
        self.end2 = nn.Conv2d(end_ch, 3, kernel_size=(1, 1))  # 3-class logits

        # regression head on end_ch features for target node at last time
        self.ret_head = nn.Linear(end_ch, 1)

        for m in self.modules():
            if isinstance(m, (nn.Conv2d, nn.Linear)):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.zeros_(m.bias)

    def _compute_supports(self, e_seq: torch.Tensor) -> Tuple[torch.Tensor, Dict[str, Any]]:
        B, L_, E, D = e_seq.shape
        e_last = e_seq[:, -1, :, :]  # (B,E,D)

        A_prior = build_adj_prior_from_edge_attr(
            edge_attr_last=e_last,
            edge_index=EDGE_INDEX.to(e_seq.device),
            n_nodes=self.n_nodes,
            use_abs=bool(self.cfg.get("prior_use_abs", False)),
            diag_boost=float(self.cfg.get("prior_diag_boost", 1.0)),
            row_normalize=bool(self.cfg.get("prior_row_normalize", True)),
        )  # (B,N,N)

        A_adapt_base, sparsity_proxy, _adapt_logits = self.adapt()  # (N,N)
        A_adapt = A_adapt_base.unsqueeze(0).expand(B, -1, -1)       # (B,N,N)

        w = self.support_mix()  # (3,)
        A_static = self.A_static.to(e_seq.device).to(e_seq.dtype).unsqueeze(0).expand(B, -1, -1)

        A_mix = w[0] * A_static + w[1] * A_prior + w[2] * A_adapt
        A_mix = A_mix / (A_mix.sum(dim=-1, keepdim=True) + 1e-8)

        # adjacency regularizers (adapt only, off-diagonal)
        N = self.n_nodes
        offdiag = (1.0 - torch.eye(N, device=e_seq.device, dtype=e_seq.dtype))
        l1_off = (sparsity_proxy.to(e_seq.dtype) * offdiag).abs().mean()
        mse_prior = ((A_adapt - A_prior) ** 2 * offdiag).mean()

        aux = {
            "support_w": w.detach().cpu().numpy().tolist(),
            "l1_off": float(l1_off.detach().cpu().item()),
            "mse_prior": float(mse_prior.detach().cpu().item()),
            "_l1_off_t": l1_off,
            "_mse_prior_t": mse_prior,
        }
        return A_mix, aux

    def forward(self, x_seq: torch.Tensor, e_seq: torch.Tensor, return_aux: bool = False):
        x_seq = torch.nan_to_num(x_seq, nan=0.0, posinf=0.0, neginf=0.0)
        e_seq = torch.nan_to_num(e_seq, nan=0.0, posinf=0.0, neginf=0.0)

        B, L_, N, Fdim = x_seq.shape
        assert N == self.n_nodes

        # (B,L,N,F) -> (B,C,N,T)
        x = self.in_proj(x_seq)              # (B,L,N,C)
        x = x.permute(0, 3, 2, 1).contiguous()  # (B,C,N,T)

        A_mix, aux = self._compute_supports(e_seq)

        skip_sum = None
        for blk in self.blocks:
            x, skip = blk(x, A_mix)
            skip_sum = skip if skip_sum is None else (skip_sum + skip)

        y = F.relu(skip_sum)
        y_end = F.relu(self.end1(y))  # (B,end_ch,N,T)
        logits_all = self.end2(y_end)  # (B,3,N,T)

        logits = logits_all[:, :, self.target_node, -1]  # (B,3)
        feat = y_end[:, :, self.target_node, -1]         # (B,end_ch)
        ret_hat = self.ret_head(feat).squeeze(-1)        # (B,)

        logits = torch.nan_to_num(logits, nan=0.0, posinf=0.0, neginf=0.0)
        ret_hat = torch.nan_to_num(ret_hat, nan=0.0, posinf=0.0, neginf=0.0)

        if return_aux:
            return logits, ret_hat, aux
        return logits, ret_hat


In [11]:
# ======================================================================
# Step 8: Metrics, PnL helpers, and multi-task loss
# ======================================================================

def _safe_auc_binary(y_true: np.ndarray, score: np.ndarray) -> float:
    y_true = np.asarray(y_true, dtype=np.int64)
    score = np.asarray(score, dtype=np.float64)
    if y_true.size == 0 or len(np.unique(y_true)) < 2:
        return float("nan")
    return float(roc_auc_score(y_true, score))


def compute_trade_dir_auc_from_probs(y_tb_true: np.ndarray, prob3: np.ndarray) -> Tuple[float, float]:
    """
    trade_auc: trade vs no-trade where trade={SHORT,LONG} vs FLAT
              score = 1 - p_flat
    dir_auc: LONG vs SHORT on true-trade samples only
             score = p_long/(p_long+p_short)
    """
    y_tb_true = np.asarray(y_tb_true, dtype=np.int64)
    prob3 = np.asarray(prob3, dtype=np.float64)

    y_trade_bin = (y_tb_true != 1).astype(np.int64)
    p_trade = 1.0 - prob3[:, 1]
    trade_auc = _safe_auc_binary(y_trade_bin, p_trade)

    mask_trade = (y_tb_true != 1)
    y_dir_bin = (y_tb_true[mask_trade] == 2).astype(np.int64)  # 1=LONG, 0=SHORT
    p_short = prob3[mask_trade, 0]
    p_long = prob3[mask_trade, 2]
    p_dir = p_long / (p_long + p_short + 1e-12)
    dir_auc = _safe_auc_binary(y_dir_bin, p_dir)

    return trade_auc, dir_auc


def pnl_from_probs_3class(prob3: np.ndarray, exit_ret_arr: np.ndarray, thr_trade: float, thr_dir: float, cost_bps: float) -> Dict[str, Any]:
    prob3 = np.asarray(prob3, dtype=np.float64)
    exit_ret_arr = np.asarray(exit_ret_arr, dtype=np.float64)

    p_short = prob3[:, 0]
    p_flat = prob3[:, 1]
    p_long = prob3[:, 2]

    trade_conf = 1.0 - p_flat
    dir_prob = p_long / (p_long + p_short + 1e-12)
    dir_conf = np.maximum(dir_prob, 1.0 - dir_prob)

    mask = (trade_conf >= float(thr_trade)) & (dir_conf >= float(thr_dir))

    action = np.zeros_like(exit_ret_arr, dtype=np.float64)
    action[mask] = np.where(dir_prob[mask] >= 0.5, 1.0, -1.0)

    cost = (float(cost_bps) * 1e-4) * mask.astype(np.float64)
    pnl = action * exit_ret_arr - cost

    n = int(len(exit_ret_arr))
    n_tr = int(mask.sum())

    return {
        "n": n,
        "n_trades": n_tr,
        "trade_rate": float(n_tr / max(1, n)),
        "pnl_sum": float(pnl.sum()),
        "pnl_mean": float(pnl.mean()) if n else float("nan"),
        "pnl_per_trade": float(pnl.sum() / max(1, n_tr)),
        "pnl_sharpe": float((pnl.mean() / (pnl.std() + 1e-12)) * np.sqrt(288)) if n else float("nan"),
    }


def build_trade_threshold_grid(p_trade: np.ndarray, base_grid: Optional[List[float]], target_trades_list: Optional[List[int]]) -> List[float]:
    p_trade = np.asarray(p_trade, dtype=np.float64)
    p_trade = p_trade[np.isfinite(p_trade)]
    if p_trade.size == 0:
        return base_grid or [0.5]

    thrs = set(float(t) for t in (base_grid or []))

    if target_trades_list:
        N = int(p_trade.size)
        for k in target_trades_list:
            k = int(k)
            if k <= 0:
                continue
            if k >= N:
                thr = float(np.min(p_trade))
            else:
                q = 1.0 - (k / N)
                thr = float(np.quantile(p_trade, q))
            thrs.add(float(np.clip(thr, 0.01, 0.99)))

    out = sorted(thrs)
    cleaned = []
    for t in out:
        if not cleaned or abs(t - cleaned[-1]) > 1e-6:
            cleaned.append(float(t))
    return cleaned


def sweep_thresholds_3class(prob3: np.ndarray, exit_ret_arr: np.ndarray, cfg: Dict[str, Any], min_trades: int, target_trade_rate: Optional[float]) -> pd.DataFrame:
    prob3 = np.asarray(prob3, dtype=np.float64)
    p_flat = prob3[:, 1]
    p_trade = 1.0 - p_flat

    thr_trade_grid = build_trade_threshold_grid(
        p_trade=p_trade,
        base_grid=cfg.get("thr_trade_grid", [0.5]),
        target_trades_list=cfg.get("proxy_target_trades", None),
    )
    thr_dir_grid = cfg.get("thr_dir_grid", [0.5])

    obj = str(cfg.get("thr_objective", "pnl_sum"))
    max_rate = cfg.get("max_trade_rate_val", None)
    penalty = float(cfg.get("trade_rate_penalty", 0.0))

    rows = []
    for thr_t in thr_trade_grid:
        for thr_d in thr_dir_grid:
            m = pnl_from_probs_3class(prob3, exit_ret_arr, thr_t, thr_d, cfg["cost_bps"])
            if int(m["n_trades"]) < int(min_trades):
                continue
            if max_rate is not None and float(m["trade_rate"]) > float(max_rate):
                continue

            base = float(m.get(obj, np.nan))
            if not np.isfinite(base):
                continue

            if target_trade_rate is not None:
                score = base - penalty * abs(float(m["trade_rate"]) - float(target_trade_rate))
            else:
                score = base - penalty * float(m["trade_rate"])

            rows.append({"thr_trade": float(thr_t), "thr_dir": float(thr_d), "score": float(score), **m})

    if not rows:
        return sweep_thresholds_3class(prob3, exit_ret_arr, cfg, min_trades=1, target_trade_rate=target_trade_rate)

    return pd.DataFrame(rows).sort_values(["score", "pnl_sum"], ascending=False)


def total_loss_with_adj_reg(loss: torch.Tensor, aux: Dict[str, Any], cfg: Dict[str, Any]) -> torch.Tensor:
    lam_l1 = float(cfg.get("adj_l1_lambda", 0.0))
    lam_pr = float(cfg.get("adj_prior_lambda", 0.0))
    reg = 0.0
    if lam_l1 > 0:
        reg = reg + lam_l1 * aux["_l1_off_t"]
    if lam_pr > 0:
        reg = reg + lam_pr * aux["_mse_prior_t"]
    return loss + reg


def multitask_loss(
    logits: torch.Tensor,
    ret_hat: torch.Tensor,
    y: torch.Tensor,
    exit_ret_batch: torch.Tensor,
    ce_loss_fn: nn.Module,
    cfg: Dict[str, Any],
) -> Tuple[torch.Tensor, Dict[str, torch.Tensor]]:
    """
    Total = w_ce * CE + w_ret * Huber(ret_hat, exit_ret) + w_util * (-utility)
    utility uses a soft position derived from probs:
      pos = trade_conf * tanh(k*(p_long - p_short))
      utility = pos * exit_ret - fee * |pos|
    """
    w_ce = float(cfg["loss_w_ce"])
    w_ret = float(cfg["loss_w_ret"])
    w_ut = float(cfg["loss_w_utility"])

    # CE
    ce = ce_loss_fn(logits, y)

    # Clip exit_ret for stability in ret/utility terms
    clip_val = float(cfg.get("exit_ret_clip", 0.0))
    if clip_val and clip_val > 0:
        er = torch.clamp(exit_ret_batch, -clip_val, clip_val)
    else:
        er = exit_ret_batch

    # Regression (Huber)
    delta = float(cfg.get("ret_huber_delta", 0.01))
    huber = F.huber_loss(ret_hat, er, delta=delta)

    # Soft utility
    probs = torch.softmax(logits, dim=-1)
    p_short = probs[:, 0]
    p_flat = probs[:, 1]
    p_long = probs[:, 2]

    trade_conf = 1.0 - p_flat
    k = float(cfg.get("utility_k", 2.0))
    pos = trade_conf * torch.tanh(k * (p_long - p_short))  # [-1,1] approximately

    fee = float(cfg.get("cost_bps", 0.0)) * 1e-4
    utility = pos * er - fee * torch.abs(pos)
    util_loss = -utility.mean()

    total = w_ce * ce + w_ret * huber + w_ut * util_loss

    parts = {
        "ce": ce.detach(),
        "huber": huber.detach(),
        "util": utility.mean().detach(),       # mean utility (higher is better)
        "util_loss": util_loss.detach(),
        "pos_abs": torch.abs(pos).mean().detach(),
    }
    return total, parts


In [12]:
# ======================================================================
# Step 9: Evaluation helpers (multitask)
# ======================================================================

@torch.no_grad()
def eval_multitask_on_indices(
    model: nn.Module,
    X_scaled: np.ndarray,
    edge_scaled: np.ndarray,
    indices: np.ndarray,
    ce_loss_fn: nn.Module,
    cfg: Dict[str, Any],
) -> Dict[str, Any]:
    ds = LobGraphSequenceDataset3Class(
        X_node=X_scaled,
        E_feat=edge_scaled,
        y_tb_arr=y_tb,
        exit_ret_arr=exit_ret,
        sample_t_=sample_t,
        indices=indices.astype(np.int64),
        lookback=cfg["lookback"],
    )
    loader = DataLoader(ds, batch_size=int(cfg["batch_size"]), shuffle=False, collate_fn=collate_fn_3class, num_workers=0)

    model.eval()
    tot_loss = 0.0
    tot_ce = 0.0
    tot_huber = 0.0
    tot_util = 0.0
    tot_pos_abs = 0.0
    n = 0

    probs_all = []
    y_all = []
    er_all = []
    ret_hat_all = []

    for x, e, y, er, _sidx in loader:
        x = x.to(DEVICE).float()
        e = e.to(DEVICE).float()
        y = y.to(DEVICE).long()
        er = er.to(DEVICE).float()

        logits, ret_hat, aux = model(x, e, return_aux=True)
        loss, parts = multitask_loss(logits, ret_hat, y, er, ce_loss_fn, cfg)
        loss = total_loss_with_adj_reg(loss, aux, cfg)

        B = int(y.size(0))
        tot_loss += float(loss.item()) * B
        tot_ce += float(parts["ce"].item()) * B
        tot_huber += float(parts["huber"].item()) * B
        tot_util += float(parts["util"].item()) * B
        tot_pos_abs += float(parts["pos_abs"].item()) * B
        n += B

        p = torch.softmax(logits, dim=-1).detach().cpu().numpy()
        probs_all.append(p)
        y_all.append(y.detach().cpu().numpy())
        er_all.append(er.detach().cpu().numpy())
        ret_hat_all.append(ret_hat.detach().cpu().numpy())

    prob3 = np.concatenate(probs_all, axis=0) if probs_all else np.zeros((0, 3), dtype=np.float64)
    y_np = np.concatenate(y_all, axis=0) if y_all else np.zeros((0,), dtype=np.int64)
    er_np = np.concatenate(er_all, axis=0) if er_all else np.zeros((0,), dtype=np.float64)
    rh_np = np.concatenate(ret_hat_all, axis=0) if ret_hat_all else np.zeros((0,), dtype=np.float64)

    trade_auc, dir_auc = compute_trade_dir_auc_from_probs(y_np, prob3)

    y_pred = prob3.argmax(axis=1) if len(y_np) else np.array([], dtype=np.int64)
    acc = float(accuracy_score(y_np, y_pred)) if len(y_np) else float("nan")
    f1m = float(f1_score(y_np, y_pred, average="macro")) if len(y_np) else float("nan")
    cm = confusion_matrix(y_np, y_pred, labels=[0, 1, 2]) if len(y_np) else None

    out = {
        "loss": float(tot_loss / max(1, n)),
        "loss_ce": float(tot_ce / max(1, n)),
        "loss_huber": float(tot_huber / max(1, n)),
        "util_mean": float(tot_util / max(1, n)),
        "pos_abs_mean": float(tot_pos_abs / max(1, n)),
        "acc": acc,
        "f1m": f1m,
        "cm": cm,
        "trade_auc": float(trade_auc) if np.isfinite(trade_auc) else float("nan"),
        "dir_auc": float(dir_auc) if np.isfinite(dir_auc) else float("nan"),
        "prob3": prob3,
        "y": y_np,
        "er": er_np,
        "ret_hat": rh_np,
    }
    return out


In [13]:
# ======================================================================
# Step 10: Artifact saving/loading (weights .pt + scaler .npz + meta .json)
# ======================================================================

def _to_jsonable_cfg(cfg: Dict[str, Any]) -> Dict[str, Any]:
    """
    Ensure cfg is JSON-serializable (paths -> str, lists -> lists).
    """
    out = {}
    for k, v in cfg.items():
        if isinstance(v, Path):
            out[k] = str(v)
        else:
            out[k] = v
    return out


def save_scaler_npz(path: Path, params: Dict[str, Any]) -> None:
    path.parent.mkdir(parents=True, exist_ok=True)
    np.savez_compressed(
        str(path),
        center_=np.asarray(params["center_"], dtype=np.float32),
        scale_=np.asarray(params["scale_"], dtype=np.float32),
        max_abs=np.asarray([float(params["max_abs"])], dtype=np.float32),
    )


def load_scaler_npz(path: Path) -> Dict[str, Any]:
    data = np.load(str(path))
    return {
        "center_": data["center_"].astype(np.float32),
        "scale_": data["scale_"].astype(np.float32),
        "max_abs": float(data["max_abs"][0]),
    }


def save_bundle(
    bundle_dir: Path,
    name: str,
    model_state: Dict[str, torch.Tensor],
    cfg: Dict[str, Any],
    node_scaler_params: Dict[str, Any],
    edge_scaler_params: Optional[Dict[str, Any]],
    extra_meta: Dict[str, Any],
) -> Dict[str, Path]:
    """
    Saves:
      - weights: {name}_weights.pt
      - scalers: {name}_node_scaler.npz and {name}_edge_scaler.npz
      - meta:    {name}_meta.json
    """
    bundle_dir.mkdir(parents=True, exist_ok=True)
    weights_path = bundle_dir / f"{name}_weights.pt"
    node_scaler_path = bundle_dir / f"{name}_node_scaler.npz"
    edge_scaler_path = bundle_dir / f"{name}_edge_scaler.npz"
    meta_path = bundle_dir / f"{name}_meta.json"

    # weights only
    torch.save(model_state, str(weights_path))

    # scalers (no pickle)
    save_scaler_npz(node_scaler_path, node_scaler_params)
    if edge_scaler_params is not None:
        save_scaler_npz(edge_scaler_path, edge_scaler_params)
        edge_scaler_file = edge_scaler_path.name
    else:
        edge_scaler_file = None

    meta = {
        "name": name,
        "weights_file": weights_path.name,
        "node_scaler_file": node_scaler_path.name,
        "edge_scaler_file": edge_scaler_file,
        "cfg": _to_jsonable_cfg(cfg),
        **extra_meta,
    }
    with open(meta_path, "w", encoding="utf-8") as f:
        json.dump(meta, f, indent=2)

    return {
        "weights": weights_path,
        "node_scaler": node_scaler_path,
        "edge_scaler": edge_scaler_path if edge_scaler_params is not None else None,
        "meta": meta_path,
    }


def load_bundle(bundle_dir: Path, name: str) -> Dict[str, Any]:
    meta_path = bundle_dir / f"{name}_meta.json"
    if not meta_path.exists():
        raise FileNotFoundError(str(meta_path))

    with open(meta_path, "r", encoding="utf-8") as f:
        meta = json.load(f)

    weights_path = bundle_dir / meta["weights_file"]
    node_scaler_path = bundle_dir / meta["node_scaler_file"]
    edge_scaler_file = meta.get("edge_scaler_file", None)
    edge_scaler_path = (bundle_dir / edge_scaler_file) if edge_scaler_file else None

    # weights-only load (safe default in torch>=2.6)
    state = torch.load(str(weights_path), map_location="cpu", weights_only=True)

    node_scaler_params = load_scaler_npz(node_scaler_path)
    edge_scaler_params = load_scaler_npz(edge_scaler_path) if edge_scaler_path else None

    return {
        "meta": meta,
        "state": state,
        "node_scaler_params": node_scaler_params,
        "edge_scaler_params": edge_scaler_params,
    }


In [14]:
# ======================================================================
# Step 11: Training one fold (multi-task), selection by trade_auc + w*dir_auc
# ======================================================================

def train_one_fold_multitask(
    fold_id: int,
    X_scaled: np.ndarray,
    edge_scaled: np.ndarray,
    idx_train: np.ndarray,
    idx_val: np.ndarray,
    idx_test: np.ndarray,
    node_scaler_params: Dict[str, Any],
    edge_scaler_params: Optional[Dict[str, Any]],
    cfg: Dict[str, Any],
) -> Dict[str, Any]:
    # Build loaders
    t_train = sample_t[idx_train]
    y_train = y_tb[t_train].astype(np.int64)

    tr_ds = LobGraphSequenceDataset3Class(X_scaled, edge_scaled, y_tb, exit_ret, sample_t, idx_train, cfg["lookback"])
    va_ds = LobGraphSequenceDataset3Class(X_scaled, edge_scaled, y_tb, exit_ret, sample_t, idx_val,   cfg["lookback"])
    te_ds = LobGraphSequenceDataset3Class(X_scaled, edge_scaled, y_tb, exit_ret, sample_t, idx_test,  cfg["lookback"])

    sampler = None
    shuffle = True
    if bool(cfg.get("use_weighted_sampler", True)):
        sampler = make_weighted_sampler_3class(y_train)
        shuffle = False

    tr_loader = DataLoader(tr_ds, batch_size=int(cfg["batch_size"]), shuffle=shuffle, sampler=sampler, collate_fn=collate_fn_3class, num_workers=0)
    va_loader = DataLoader(va_ds, batch_size=int(cfg["batch_size"]), shuffle=False, collate_fn=collate_fn_3class, num_workers=0)
    te_loader = DataLoader(te_ds, batch_size=int(cfg["batch_size"]), shuffle=False, collate_fn=collate_fn_3class, num_workers=0)

    model = GraphWaveNetMultiTask(
        node_in=int(X_scaled.shape[-1]),
        edge_dim=int(edge_scaled.shape[-1]),
        cfg=cfg,
        n_nodes=len(ASSETS),
        target_node=TARGET_NODE,
    ).to(DEVICE)

    ce_w = make_ce_weights_3class(y_train)
    ce_loss_fn = nn.CrossEntropyLoss(weight=ce_w, label_smoothing=float(cfg.get("label_smoothing", 0.0)))

    opt = torch.optim.AdamW(model.parameters(), lr=float(cfg["lr"]), weight_decay=float(cfg["weight_decay"]))

    use_onecycle = bool(cfg.get("use_onecycle", True))
    if use_onecycle:
        sch = torch.optim.lr_scheduler.OneCycleLR(
            opt,
            max_lr=float(cfg["lr"]),
            epochs=int(cfg["epochs"]),
            steps_per_epoch=max(1, len(tr_loader)),
            pct_start=0.15,
            div_factor=10.0,
            final_div_factor=50.0,
        )
    else:
        sch = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode="max", factor=0.5, patience=3)

    best_sel = -1e18
    best_state = None
    best_epoch = -1
    patience = 7
    bad = 0

    w_dir = float(cfg.get("sel_metric_dir_weight", 0.5))
    trade_pen = float(cfg.get("trade_prob_penalty", 0.0))

    def selection_metric(trade_auc: float, dir_auc: float) -> float:
        ta = float(trade_auc) if np.isfinite(trade_auc) else -1e18
        da = float(dir_auc) if np.isfinite(dir_auc) else 0.0
        return ta + w_dir * da

    for ep in range(1, int(cfg["epochs"]) + 1):
        model.train()

        tot = 0.0
        tot_ce = 0.0
        tot_huber = 0.0
        tot_util = 0.0
        n = 0

        for x, e, y, er, _sidx in tr_loader:
            x = x.to(DEVICE).float()
            e = e.to(DEVICE).float()
            y = y.to(DEVICE).long()
            er = er.to(DEVICE).float()

            opt.zero_grad(set_to_none=True)

            logits, ret_hat, aux = model(x, e, return_aux=True)

            # multi-task base loss
            loss_mt, parts = multitask_loss(logits, ret_hat, y, er, ce_loss_fn, cfg)

            # optional probability mass penalty to reduce overtrading
            if trade_pen > 0:
                p = torch.softmax(logits, dim=-1)
                p_trade = (p[:, 0] + p[:, 2]).mean()
                loss_mt = loss_mt + trade_pen * p_trade

            loss = total_loss_with_adj_reg(loss_mt, aux, cfg)

            if not torch.isfinite(loss):
                continue

            loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), float(cfg["grad_clip"]))
            opt.step()
            if use_onecycle:
                sch.step()

            B = int(y.size(0))
            tot += float(loss.item()) * B
            tot_ce += float(parts["ce"].item()) * B
            tot_huber += float(parts["huber"].item()) * B
            tot_util += float(parts["util"].item()) * B
            n += B

        tr_loss = tot / max(1, n)
        tr_ce = tot_ce / max(1, n)
        tr_huber = tot_huber / max(1, n)
        tr_util = tot_util / max(1, n)

        # epoch val AUCs + val losses (for monitoring)
        model.eval()
        val_probs = []
        val_ys = []
        val_er = []

        val_tot = 0.0
        val_n = 0
        val_ce_sum = 0.0
        val_huber_sum = 0.0
        val_util_sum = 0.0

        for x, e, y, er, _sidx in va_loader:
            x = x.to(DEVICE).float()
            e = e.to(DEVICE).float()
            y = y.to(DEVICE).long()
            er = er.to(DEVICE).float()

            logits, ret_hat, aux = model(x, e, return_aux=True)
            loss_mt, parts = multitask_loss(logits, ret_hat, y, er, ce_loss_fn, cfg)
            loss_val = total_loss_with_adj_reg(loss_mt, aux, cfg)

            B = int(y.size(0))
            val_tot += float(loss_val.item()) * B
            val_ce_sum += float(parts["ce"].item()) * B
            val_huber_sum += float(parts["huber"].item()) * B
            val_util_sum += float(parts["util"].item()) * B
            val_n += B

            p = torch.softmax(logits, dim=-1).detach().cpu().numpy()
            val_probs.append(p)
            val_ys.append(y.detach().cpu().numpy())
            val_er.append(er.detach().cpu().numpy())

        val_prob3 = np.concatenate(val_probs, axis=0) if val_probs else np.zeros((0, 3), dtype=np.float64)
        val_y = np.concatenate(val_ys, axis=0) if val_ys else np.zeros((0,), dtype=np.int64)

        trade_auc, dir_auc = compute_trade_dir_auc_from_probs(val_y, val_prob3)
        sel = selection_metric(trade_auc, dir_auc)

        val_loss = val_tot / max(1, val_n)
        val_ce = val_ce_sum / max(1, val_n)
        val_huber = val_huber_sum / max(1, val_n)
        val_util = val_util_sum / max(1, val_n)

        if sel > best_sel:
            best_sel = sel
            best_epoch = ep
            best_state = {k: v.detach().cpu().clone() for k, v in model.state_dict().items()}
            bad = 0
        else:
            bad += 1

        if not use_onecycle:
            sch.step(sel)

        lr_now = opt.param_groups[0]["lr"]
        w_support = model.support_mix().detach().cpu().numpy().tolist()

        print(
            f"[fold {fold_id:02d}] ep {ep:02d} lr={lr_now:.2e} "
            f"tr_loss={tr_loss:.4f} (ce={tr_ce:.4f}, huber={tr_huber:.4f}, util={tr_util:.5f}) "
            f"val_loss={val_loss:.4f} (ce={val_ce:.4f}, huber={val_huber:.4f}, util={val_util:.5f}) "
            f"val_trade_auc={trade_auc:.3f} val_dir_auc={dir_auc:.3f} sel={sel:.3f} "
            f"best={best_sel:.3f}@ep{best_epoch:02d} supports={np.round(w_support, 3).tolist()}"
        )

        if bad >= patience:
            break

    if best_state is not None:
        model.load_state_dict(best_state)

    # Eval (full) on VAL and TEST
    val_eval = eval_multitask_on_indices(model, X_scaled, edge_scaled, idx_val, ce_loss_fn, cfg)
    test_eval = eval_multitask_on_indices(model, X_scaled, edge_scaled, idx_test, ce_loss_fn, cfg)

    # thresholds chosen on VAL for PnL, then applied to TEST
    true_val_trade_rate = split_trade_ratio(idx_val, sample_t, y_trade)
    sweep_val = sweep_thresholds_3class(
        prob3=val_eval["prob3"],
        exit_ret_arr=val_eval["er"],
        cfg=cfg,
        min_trades=int(cfg["eval_min_trades"]),
        target_trade_rate=float(true_val_trade_rate),
    )
    best_thr = sweep_val.iloc[0].to_dict()
    thr_trade = float(best_thr["thr_trade"])
    thr_dir = float(best_thr["thr_dir"])

    pnl_val = pnl_from_probs_3class(val_eval["prob3"], val_eval["er"], thr_trade, thr_dir, cfg["cost_bps"])
    pnl_test = pnl_from_probs_3class(test_eval["prob3"], test_eval["er"], thr_trade, thr_dir, cfg["cost_bps"])

    print(
        f"[fold {fold_id:02d}] chosen thresholds on VAL: thr_trade={thr_trade:.3f} thr_dir={thr_dir:.3f} "
        f"| val pnl_sum={pnl_val['pnl_sum']:.4f} val trade_rate={pnl_val['trade_rate']:.3f}"
    )
    print(
        f"[fold {fold_id:02d}] TEST (fixed thresholds from VAL): "
        f"trade_auc={test_eval['trade_auc']:.3f} dir_auc={test_eval['dir_auc']:.3f} "
        f"pnl_sum={pnl_test['pnl_sum']:.4f} trade_rate={pnl_test['trade_rate']:.3f} trades={pnl_test['n_trades']}"
    )

    return {
        "fold": int(fold_id),
        "model_state": {k: v.detach().cpu().clone() for k, v in model.state_dict().items()},
        "node_scaler_params": node_scaler_params,
        "edge_scaler_params": edge_scaler_params,
        "idx_train": idx_train,
        "idx_val": idx_val,
        "idx_test": idx_test,
        "best_epoch": int(best_epoch),
        "best_sel": float(best_sel),
        "val_eval": val_eval,
        "test_eval": test_eval,
        "thr_trade": thr_trade,
        "thr_dir": thr_dir,
        "pnl_val": pnl_val,
        "pnl_test": pnl_test,
        "sweep_val_head": sweep_val.head(5),
    }


In [15]:
# ======================================================================
# Step 12: Walk-forward CV run + saving per-fold bundles + overall best
# ======================================================================

def run_walk_forward_cv_multitask() -> Tuple[pd.DataFrame, List[Dict[str, Any]], str]:
    fold_artifacts: List[Dict[str, Any]] = []
    rows: List[Dict[str, Any]] = []

    best_overall_sel = -1e18
    best_overall_name = None

    for fi, (idx_tr, idx_va, idx_te) in enumerate(walk_splits, 1):
        print("\n" + "=" * 90)
        print(f"FOLD {fi}/{len(walk_splits)} sizes: train={len(idx_tr)} val={len(idx_va)} test={len(idx_te)}")
        print(f"True trade ratio (val):  {split_trade_ratio(idx_va, sample_t, y_trade):.3f}")
        print(f"True trade ratio (test): {split_trade_ratio(idx_te, sample_t, y_trade):.3f}")

        # Fold scaling (fit only on fold train timeline)
        X_scaled, node_params = fit_scale_nodes_train_only(X_node_raw, sample_t, idx_tr, max_abs=CFG["max_abs_feat"])
        if bool(CFG.get("edge_scale", True)):
            edge_scaled, edge_params = fit_scale_edges_train_only(edge_feat, sample_t, idx_tr, max_abs=CFG["max_abs_edge"])
        else:
            edge_scaled = edge_feat.astype(np.float32)
            edge_params = None

        artifact = train_one_fold_multitask(
            fold_id=fi,
            X_scaled=X_scaled,
            edge_scaled=edge_scaled,
            idx_train=idx_tr,
            idx_val=idx_va,
            idx_test=idx_te,
            node_scaler_params=node_params,
            edge_scaler_params=edge_params,
            cfg=CFG,
        )

        # Save fold bundle
        fold_name = f"fold_{fi:02d}"
        extra_meta = {
            "kind": "fold_best",
            "fold": fi,
            "best_epoch": artifact["best_epoch"],
            "best_sel": artifact["best_sel"],
            "thr_trade": artifact["thr_trade"],
            "thr_dir": artifact["thr_dir"],
            "idx_train": artifact["idx_train"].tolist(),
            "idx_val": artifact["idx_val"].tolist(),
            "idx_test": artifact["idx_test"].tolist(),
        }
        saved = save_bundle(
            bundle_dir=ART_DIR,
            name=fold_name,
            model_state=artifact["model_state"],
            cfg=CFG,
            node_scaler_params=artifact["node_scaler_params"],
            edge_scaler_params=artifact["edge_scaler_params"],
            extra_meta=extra_meta,
        )
        print("Saved fold bundle:", saved["meta"].name)

        if float(artifact["best_sel"]) > best_overall_sel:
            best_overall_sel = float(artifact["best_sel"])
            best_overall_name = fold_name

        fold_artifacts.append(artifact)

        rows.append({
            "fold": fi,
            "val_trade_auc": artifact["val_eval"]["trade_auc"],
            "val_dir_auc": artifact["val_eval"]["dir_auc"],
            "val_loss": artifact["val_eval"]["loss"],
            "test_trade_auc": artifact["test_eval"]["trade_auc"],
            "test_dir_auc": artifact["test_eval"]["dir_auc"],
            "test_loss": artifact["test_eval"]["loss"],
            "thr_trade": artifact["thr_trade"],
            "thr_dir": artifact["thr_dir"],
            "test_trade_rate_pred": artifact["pnl_test"]["trade_rate"],
            "test_pnl_sum": artifact["pnl_test"]["pnl_sum"],
            "test_pnl_mean": artifact["pnl_test"]["pnl_mean"],
            "test_n_trades": artifact["pnl_test"]["n_trades"],
            "best_sel": artifact["best_sel"],
        })

    cv_summary = pd.DataFrame(rows)

    # Create an "overall_best" alias by copying the best fold files
    assert best_overall_name is not None
    overall_name = "overall_best"
    best_bundle = load_bundle(ART_DIR, best_overall_name)

    # Copy weights + scalers by re-saving under overall_best
    extra_meta = {
        "kind": "overall_best",
        "source_name": best_overall_name,
        "source_fold": best_bundle["meta"].get("fold", None),
        "thr_trade": best_bundle["meta"]["thr_trade"],
        "thr_dir": best_bundle["meta"]["thr_dir"],
        "idx_train": best_bundle["meta"]["idx_train"],
        "idx_val": best_bundle["meta"]["idx_val"],
        "idx_test": best_bundle["meta"]["idx_test"],
    }
    save_bundle(
        bundle_dir=ART_DIR,
        name=overall_name,
        model_state=best_bundle["state"],
        cfg=CFG,
        node_scaler_params=best_bundle["node_scaler_params"],
        edge_scaler_params=best_bundle["edge_scaler_params"],
        extra_meta=extra_meta,
    )
    print("\nSaved overall best bundle as:", overall_name)

    return cv_summary, fold_artifacts, overall_name


cv_summary_mt, fold_artifacts_mt, overall_best_name = run_walk_forward_cv_multitask()

print("\n" + "=" * 90)
print("CV summary (multi-task; TEST uses thresholds selected on VAL):")
print(cv_summary_mt)
print("\nMeans:")
print(cv_summary_mt.mean(numeric_only=True))



FOLD 1/4 sizes: train=5652 val=1130 test=1130
True trade ratio (val):  0.365
True trade ratio (test): 0.304
[fold 01] ep 01 lr=9.84e-05 tr_loss=1.0414 (ce=1.0329, huber=0.0024, util=-0.00002) val_loss=1.1555 (ce=1.1546, huber=0.0019, util=0.00002) val_trade_auc=0.517 val_dir_auc=0.433 sel=0.734 best=0.734@ep01 supports=[0.333, 0.334, 0.333]
[fold 01] ep 02 lr=2.34e-04 tr_loss=0.9871 (ce=0.9776, huber=0.0023, util=0.00012) val_loss=1.2073 (ce=1.2065, huber=0.0017, util=0.00008) val_trade_auc=0.524 val_dir_auc=0.454 sel=0.751 best=0.751@ep02 supports=[0.333, 0.334, 0.333]
[fold 01] ep 03 lr=3.00e-04 tr_loss=0.9616 (ce=0.9521, huber=0.0020, util=0.00029) val_loss=1.2228 (ce=1.2220, huber=0.0015, util=0.00003) val_trade_auc=0.519 val_dir_auc=0.462 sel=0.750 best=0.751@ep02 supports=[0.334, 0.332, 0.334]
[fold 01] ep 04 lr=2.97e-04 tr_loss=0.9232 (ce=0.9137, huber=0.0018, util=0.00051) val_loss=1.2143 (ce=1.2135, huber=0.0014, util=0.00003) val_trade_auc=0.518 val_dir_auc=0.527 sel=0.782 b

In [16]:
# ======================================================================
# Step 13: Evaluate a saved bundle on arbitrary indices (e.g., FINAL holdout)
# ======================================================================

@torch.no_grad()
def evaluate_bundle_on_indices(bundle_dir: Path, name: str, indices: np.ndarray, label: str) -> Dict[str, Any]:
    bundle = load_bundle(bundle_dir, name)

    # rebuild model
    cfg = bundle["meta"]["cfg"]
    model = GraphWaveNetMultiTask(
        node_in=int(X_node_raw.shape[-1]),
        edge_dim=int(edge_feat.shape[-1]),
        cfg=cfg,
        n_nodes=len(ASSETS),
        target_node=TARGET_NODE,
    ).to(DEVICE)
    model.load_state_dict(bundle["state"])
    model.eval()

    # apply scalers
    X_scaled = apply_scaler_params(X_node_raw.astype(np.float32), bundle["node_scaler_params"])
    if bundle["edge_scaler_params"] is not None:
        E_scaled = apply_scaler_params(edge_feat.astype(np.float32), bundle["edge_scaler_params"])
    else:
        E_scaled = edge_feat.astype(np.float32)

    # CE weights from saved train split labels (approximate)
    idx_train_saved = np.asarray(bundle["meta"]["idx_train"], dtype=np.int64)
    t_train = sample_t[idx_train_saved]
    y_train = y_tb[t_train].astype(np.int64)
    ce_w = make_ce_weights_3class(y_train)
    ce_loss_fn = nn.CrossEntropyLoss(weight=ce_w, label_smoothing=float(CFG.get("label_smoothing", 0.0)))

    ev = eval_multitask_on_indices(model, X_scaled, E_scaled, indices.astype(np.int64), ce_loss_fn, cfg)

    thr_trade = float(bundle["meta"]["thr_trade"])
    thr_dir = float(bundle["meta"]["thr_dir"])
    pnl = pnl_from_probs_3class(ev["prob3"], ev["er"], thr_trade, thr_dir, float(cfg["cost_bps"]))

    print("\n" + "=" * 90)
    print(label)
    print(f"bundle: {name}")
    print(f"trade_auc={ev['trade_auc']:.3f} | dir_auc={ev['dir_auc']:.3f} | val/test loss={ev['loss']:.4f}")
    print(f"soft util mean={ev['util_mean']:.6f} | pos_abs mean={ev['pos_abs_mean']:.4f}")
    print(f"pnl_sum={pnl['pnl_sum']:.4f} | trade_rate={pnl['trade_rate']:.3f} | trades={pnl['n_trades']}")
    return {"eval": ev, "pnl": pnl}


# Evaluate overall best on FINAL holdout (10%) without refit
holdout_indices = idx_final_test.astype(np.int64)
_ = evaluate_bundle_on_indices(ART_DIR, overall_best_name, holdout_indices, label="FINAL HOLDOUT (10%) using overall_best")



FINAL HOLDOUT (10%) using overall_best
bundle: overall_best
trade_auc=0.536 | dir_auc=0.500 | val/test loss=1.1690
soft util mean=-0.000278 | pos_abs mean=0.3416
pnl_sum=-0.6337 | trade_rate=0.494 | trades=621


In [17]:
# ======================================================================
# Step 14: Production fit on CV(90%) -> select thresholds on val_final -> eval holdout(10%)
# ======================================================================

def production_fit_and_save() -> str:
    print("\n" + "=" * 90)
    print("PRODUCTION FIT: train on CV(90%) -> select thresholds on val_final -> eval on FINAL holdout(10%)")

    val_w = max(1, int(CFG["val_window_frac"] * n_samples_cv))
    train_end = n_samples_cv - val_w

    idx_train_final = np.arange(0, train_end, dtype=np.int64)
    idx_val_final = np.arange(train_end, n_samples_cv, dtype=np.int64)
    idx_holdout = idx_final_test.astype(np.int64)

    print("Sizes:")
    print("  train_final:", len(idx_train_final))
    print("  val_final  :", len(idx_val_final))
    print("  holdout    :", len(idx_holdout))
    print(f"True trade ratio (val_final): {split_trade_ratio(idx_val_final, sample_t, y_trade):.3f}")
    print(f"True trade ratio (holdout):   {split_trade_ratio(idx_holdout, sample_t, y_trade):.3f}")

    X_scaled, node_params = fit_scale_nodes_train_only(X_node_raw, sample_t, idx_train_final, max_abs=CFG["max_abs_feat"])
    if bool(CFG.get("edge_scale", True)):
        edge_scaled, edge_params = fit_scale_edges_train_only(edge_feat, sample_t, idx_train_final, max_abs=CFG["max_abs_edge"])
    else:
        edge_scaled = edge_feat.astype(np.float32)
        edge_params = None

    artifact = train_one_fold_multitask(
        fold_id=99,
        X_scaled=X_scaled,
        edge_scaled=edge_scaled,
        idx_train=idx_train_final,
        idx_val=idx_val_final,
        idx_test=idx_holdout,
        node_scaler_params=node_params,
        edge_scaler_params=edge_params,
        cfg=CFG,
    )

    prod_name = "production_best"
    extra_meta = {
        "kind": "production_best",
        "fold": 99,
        "best_epoch": artifact["best_epoch"],
        "best_sel": artifact["best_sel"],
        "thr_trade": artifact["thr_trade"],
        "thr_dir": artifact["thr_dir"],
        "idx_train": artifact["idx_train"].tolist(),
        "idx_val": artifact["idx_val"].tolist(),
        "idx_test": artifact["idx_test"].tolist(),  # holdout
    }
    save_bundle(
        bundle_dir=ART_DIR,
        name=prod_name,
        model_state=artifact["model_state"],
        cfg=CFG,
        node_scaler_params=artifact["node_scaler_params"],
        edge_scaler_params=artifact["edge_scaler_params"],
        extra_meta=extra_meta,
    )
    print("\nSaved production bundle as:", prod_name)
    return prod_name


production_name = production_fit_and_save()

# Evaluate production bundle on its saved idx_test (holdout)
prod_bundle = load_bundle(ART_DIR, production_name)
idx_eval = np.asarray(prod_bundle["meta"]["idx_test"], dtype=np.int64)
_ = evaluate_bundle_on_indices(ART_DIR, production_name, idx_eval, label="TEST-ONLY FROM PRODUCTION BUNDLE (holdout)")



PRODUCTION FIT: train on CV(90%) -> select thresholds on val_final -> eval on FINAL holdout(10%)
Sizes:
  train_final: 10175
  val_final  : 1130
  holdout    : 1256
True trade ratio (val_final): 0.504
True trade ratio (holdout):   0.576
[fold 99] ep 01 lr=9.80e-05 tr_loss=1.0672 (ce=1.0589, huber=0.0020, util=0.00002) val_loss=1.1575 (ce=1.1567, huber=0.0018, util=0.00005) val_trade_auc=0.572 val_dir_auc=0.513 sel=0.828 best=0.828@ep01 supports=[0.333, 0.333, 0.333]
[fold 99] ep 02 lr=2.34e-04 tr_loss=1.0018 (ce=0.9924, huber=0.0019, util=0.00012) val_loss=1.1577 (ce=1.1569, huber=0.0016, util=0.00005) val_trade_auc=0.598 val_dir_auc=0.531 sel=0.863 best=0.863@ep02 supports=[0.334, 0.332, 0.334]
[fold 99] ep 03 lr=3.00e-04 tr_loss=0.9750 (ce=0.9657, huber=0.0017, util=0.00028) val_loss=1.1783 (ce=1.1776, huber=0.0014, util=0.00010) val_trade_auc=0.611 val_dir_auc=0.536 sel=0.879 best=0.879@ep03 supports=[0.334, 0.332, 0.334]
[fold 99] ep 04 lr=2.97e-04 tr_loss=0.9454 (ce=0.9361, huber

In [18]:
# ======================================================================
# Step 15: Sanity check: load a bundle and do a single forward pass
# ======================================================================

bundle_dbg = load_bundle(ART_DIR, overall_best_name)
cfg_dbg = bundle_dbg["meta"]["cfg"]

model_dbg = GraphWaveNetMultiTask(
    node_in=int(X_node_raw.shape[-1]),
    edge_dim=int(edge_feat.shape[-1]),
    cfg=cfg_dbg,
    n_nodes=len(ASSETS),
    target_node=TARGET_NODE,
).to(DEVICE)
model_dbg.load_state_dict(bundle_dbg["state"])
model_dbg.eval()

X_dbg = apply_scaler_params(X_node_raw.astype(np.float32), bundle_dbg["node_scaler_params"])
E_dbg = apply_scaler_params(edge_feat.astype(np.float32), bundle_dbg["edge_scaler_params"]) if bundle_dbg["edge_scaler_params"] is not None else edge_feat.astype(np.float32)

ds_dbg = LobGraphSequenceDataset3Class(X_dbg, E_dbg, y_tb, exit_ret, sample_t, np.arange(0, 2, dtype=np.int64), CFG["lookback"])
x_seq, e_seq, y0, er0, sidx0 = ds_dbg[0]

with torch.no_grad():
    logits, ret_hat = model_dbg(x_seq.unsqueeze(0).to(DEVICE).float(), e_seq.unsqueeze(0).to(DEVICE).float(), return_aux=False)

print("Single sample shapes:")
print("  x_seq:", tuple(x_seq.shape), "(L,N,F)")
print("  e_seq:", tuple(e_seq.shape), "(L,E,D)")
print("Forward outputs:")
print("  logits:", tuple(logits.shape), "expected (1,3)")
print("  ret_hat:", tuple(ret_hat.shape), "expected (1,)")
print("  y_tb:", int(y0.item()), "| exit_ret:", float(er0.item()))


Single sample shapes:
  x_seq: (240, 3, 15) (L,N,F)
  e_seq: (240, 9, 20) (L,E,D)
Forward outputs:
  logits: (1, 3) expected (1,3)
  ret_hat: (1,) expected (1,)
  y_tb: 0 | exit_ret: -0.008445847779512405
